In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import torch

import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from sklearn.preprocessing import StandardScaler    
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
#generating 10k points 
np.random.seed(0)
data = 2*np.random.uniform(size=(10000, 2)) - 1

In [3]:
plt.rcParams['figure.figsize'] = 10, 8
#point labelling function
def point_labeller(a):
  if(a[0]>0 and a[1]>0) or (a[0]< 0 and a[1]< 0):
    return 1
  elif(a[0]==0 and a[1]==0):
    return -1
  else:
    return 0

#list to hold labelled data
labelled_data = []

for i in range(len(data)):
  k = list(data[i])
  #third coordinate will be the label
  labelled_data += [k + [point_labeller(k)]]

In [4]:
data = pd.DataFrame(labelled_data)
data.columns = ['X','Y','Label']

In [5]:
X = data[['X','Y']]
y = data[['Label']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [6]:
X_valid,X_test, y_valid,y_test = train_test_split(X_test,y_test,test_size=0.5)

In [7]:
#Normalize input
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)
X_valid = scaler.fit_transform(X_valid)

In [8]:
EPOCHS = 100
BATCH = 16
LR = 1e-3

In [9]:
class MyDatatype(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)


In [10]:
train_data = MyDatatype(torch.Tensor(X_train), torch.Tensor(np.array(y_train)))
test_data = MyDatatype(torch.Tensor(X_test), torch.Tensor(np.array(y_test)))
valid_data = MyDatatype(torch.Tensor(X_valid), torch.Tensor(np.array(y_valid)))


In [11]:
train_loader = DataLoader(dataset=train_data, batch_size=BATCH, shuffle=True,drop_last=True)
test_loader = DataLoader(dataset=test_data, batch_size=BATCH,drop_last=True,shuffle=True)

In [12]:
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.layer_1 = nn.Linear(2, 64) 
        self.layer_2 = nn.Linear(64, 64)
        self.layer_out = nn.Linear(64, 1) 
        
        self.relu = nn.ReLU()
        self.batchnorm1 = nn.BatchNorm1d(64)
        self.batchnorm2 = nn.BatchNorm1d(64)
        
    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.batchnorm1(x)
        x = self.relu(self.layer_2(x))
        x = self.layer_out(x)
        
        return x

In [13]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = NeuralNet()
model.to(device)
print(model)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model.parameters(), lr=LR)

NeuralNet(
  (layer_1): Linear(in_features=2, out_features=64, bias=True)
  (layer_2): Linear(in_features=64, out_features=64, bias=True)
  (layer_out): Linear(in_features=64, out_features=1, bias=True)
  (relu): ReLU()
  (batchnorm1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


In [14]:
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    
    return acc

In [15]:
model.train()
stats = []
for e in range(1, EPOCHS+1):
    stat = []
    epoch_loss = 0
    epoch_acc = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        acc = binary_acc(y_pred, y_batch)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_acc += acc.item()
    if(e%10==0):
        print(f'Epoch {e}: | Train_Loss: {epoch_loss/len(train_loader):.5f} | Train_Acc: {epoch_acc/len(train_loader):.3f}')
    stat += [e,epoch_loss/len(train_loader),epoch_acc/len(train_loader)]    
    epoch_loss = 0
    epoch_acc = 0
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        with torch.no_grad():
            y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        acc = binary_acc(y_pred, y_batch)
        epoch_loss += loss.item()
        epoch_acc += acc.item()
    if(e%10==0):
        print(f'          | Valid_Loss: {epoch_loss/len(train_loader):.5f} | Valid_Acc: {epoch_acc/len(train_loader):.3f}')
    stat += [epoch_loss/len(train_loader),epoch_acc/len(train_loader)]
    stats += [stat]    

Epoch 10: | Train_Loss: 0.28001 | Train_Acc: 90.497
          | Valid_Loss: 0.06214 | Valid_Acc: 19.094
Epoch 20: | Train_Loss: 0.22556 | Train_Acc: 91.529
          | Valid_Loss: 0.04728 | Valid_Acc: 19.460
Epoch 30: | Train_Loss: 0.21754 | Train_Acc: 91.318
          | Valid_Loss: 0.04331 | Valid_Acc: 19.638
Epoch 40: | Train_Loss: 0.20228 | Train_Acc: 91.760
          | Valid_Loss: 0.04788 | Valid_Acc: 19.252
Epoch 50: | Train_Loss: 0.18896 | Train_Acc: 92.000
          | Valid_Loss: 0.04207 | Valid_Acc: 19.558
Epoch 60: | Train_Loss: 0.18452 | Train_Acc: 92.801
          | Valid_Loss: 0.04078 | Valid_Acc: 19.854
Epoch 70: | Train_Loss: 0.16771 | Train_Acc: 93.030
          | Valid_Loss: 0.03908 | Valid_Acc: 19.627
Epoch 80: | Train_Loss: 0.17017 | Train_Acc: 93.426
          | Valid_Loss: 0.04039 | Valid_Acc: 19.815
Epoch 90: | Train_Loss: 0.15511 | Train_Acc: 93.799
          | Valid_Loss: 0.03563 | Valid_Acc: 19.835
Epoch 100: | Train_Loss: 0.15059 | Train_Acc: 94.227
          |

In [16]:
Stats = pd.DataFrame(columns=['Epoch','Train_Loss','Validation_Loss','Train_Accuracy','Validation_Accuracy'],data=stats)

In [17]:
display(Stats)

,Epoch,Train_Loss,Validation_Loss,Train_Accuracy,Validation_Accuracy
0,1,0.634133,67.164760,0.125978,17.947368
1,2,0.553856,88.050343,0.111113,19.048055
2,3,0.487533,89.471396,0.098421,18.919908
3,4,0.430560,90.016018,0.088314,18.931350
4,5,0.388911,89.823799,0.077873,19.288330
...,...,...,...,...,...
95,96,0.162143,93.306636,0.033406,19.981693
96,97,0.159066,93.839817,0.034593,19.887872
97,98,0.159433,93.334096,0.035982,19.743707
98,99,0.154083,93.569794,0.034987,19.812357
